# HFK Caluculator for a knot represented by grid notation

## Modules

In [ ]:
import numpy as np
import pandas as pd

from multiprocessing import Pool, cpu_count
from tqdm.notebook import tqdm

from GridPythonModule import *
from GPM_wrapper import *
from knot_floer_homology import *

## Input a knot using grid notation

In [ ]:
import ast
from ipywidgets import Text, HTML, Button, Output
from IPython.display import display, clear_output

class Grid_getter():
    def __init__(self):
        self.grid = []
        self.xprm = Text(value='',
                         placeholder='x marking 対応の置換を入力',
                         description='x の置換:',
                         disabled=False
                        )
        self.oprm = Text(value='',
                         placeholder='o marking 対応の置換を入力',
                         description='o の置換:',
                         disabled=False
                        )
        self.start()
        
    def start(self):
        self.xprm.observe(get_list_str, names='value')
        self.oprm.observe(get_list_str, names='value')
        
        button = Button(description='Get Grid')
        output = Output(layour={'border': '1px solid black'})

        def on_click_callback(clicked_button: Button) -> None:
            self.grid = []
            with output:
                self.grid = [ast.literal_eval(f"[{v}]") for v in [xprm.value, oprm.value]]
                print(f'\rGrid = {self.grid}', end="")
                try:
                    Grid(*self.grid).draw()
                except:
                    print("\nInvalid grid data")

        # ボタンにイベントハンドラを登録する
        button.on_click(on_click_callback)
        display(xprm, oprm, button, output)
            
    def get_list_str(arg): 
        return f"[{arg}]"

gg = Grid_getter()

In [ ]:
def get_list_str(arg):
    return f"[{arg}]"
    
xprm = Text(value='',
            placeholder='x marking 対応の置換を入力',
            description='x の置換:',
            disabled=False
           )
oprm = Text(value='',
            placeholder='o marking 対応の置換を入力',
            description='o の置換:',
            disabled=False
           )
xprm.observe(get_list_str, names='value')
oprm.observe(get_list_str, names='value')

grid = f"{xprm.value},{oprm.value}"
html=HTML(value="入力した grid notation は <font color=red>"+grid+"</font> ")

#display(f"[[{xprm}], [{oprm}]]")
display(xprm, oprm)
display(html)

In [ ]:
test = input()
print(f"You inputed this --> {test}")

In [ ]:
abe_knot_grid = [[8,11,10,3,7,2,5,6,0,12,9,1,4],[12,7,5,9,1,6,8,11,3,4,2,10,0]]
abe_knot = Grid(*abe_knot_grid)

abe_knot.draw()
print(f"crossing number: {len(abe_knot.crossings)}")

In [ ]:
PD = abe_knot.get_PD(); print(f"PD notation: \n {PD}\n")
hfk = pd_to_hfk(PD); print(f"hfk values: \n {hfk}\n")

'ranks' の最後が (2, 0): 1 となってますが、('maslov gr(=genus)', 'alexander gr'): rank  ですから、ラムダは 0 のようですね。

また、鏡像も見ておくと、今度は 'ranks' の最初を見て、(-2, -4): 1 とありますので、鏡像のラムダは 4 ですね。

In [ ]:
for mgr, agr in hfk['ranks'].keys():
    if mgr == hfk['seifert_genus']:
        lmd = agr
output = {'cr': 19, 'seifert_genus': mgr, 'lmd': lmd, 'ranks': hfk['ranks'], 'grid': abe_knot.orig, 'PD notation': PD}
df = pd.DataFrame(output); display(df)

## Scratch

In [ ]:
result = []
N, gsize = 100, 30

with tqdm(total=N, desc = '# of required knots') as pbar:
    while len(result) < N:
        ### Collect fitted grids
        Glist, Glist_len = [], 80
        with tqdm(total=Glist_len, desc = 'generating Glist') as pbar_Glist:
            fails, fmax = 0, 100
            while len(Glist) < Glist_len and fails < fmax:
                with Pool(processes=8) as pool:
                    grids = [generate_random_grid(gsize) for _ in range(100)]
                    output = [v for v in pool.map(get_fitted_grid, grids) if v is not None]
                    if len(output) > 0: 
                        Glist += output
                        pbar_Glist.update(len(output))
                    else:
                        fails += 1
            if fails >= fmax:
                print(f"Fail to generate Glist {fails} times!")
                break

        ### Collect knots satisfying seifert genus > lamda
        with Pool(processes=8) as pool:
            output = [v for v in list(tqdm(pool.imap(calc_lmd, Glist), total = len(Glist), desc='hfk calc')) if v is not None]
            if len(output) > 0:
                result += output
                pbar.update(len(output))
#---
DF = pd.DataFrame(result); display(DF)
DF.to_csv(f"./data/big_knots_with_small_lambda.csv", mode = 'a', index=False, header = False) #True)

In [ ]:
flag, count = True, 0
while flag:
    count += 1
    G = generate_random_grid(30);
    Glist = [G, destabilize_all(G), simplify_grid(G)]
    cnlist = list(map(lambda g: crossing_number(g), Glist))
    if cnlist[0] < cnlist[2] or cnlist[0] < cnlist[1]:
        flag = False
        for g in Glist:
            draw_grid(g); print(crossing_number(g))
    if count > 1000:
        flag = False